<a href="https://colab.research.google.com/github/falconlee236/DeepLearningFrom_Scratch/blob/main/Book_1/ch07/Chapter_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7 convolutional neural network(CNN)

**7.4 Implementing Convolution/Pooling layer**

*7.4.1 4-dimension Array*

In [ ]:
import numpy as np

# data 10, channel 1 height 28, width 28
x = np.random.rand(10, 1, 28, 28)
x.shape

In [ ]:
x[0].shape

In [ ]:
x[1].shape

In [ ]:
x[0, 0] # or x[0][0]

*7.4.3 Implementing Convolutional layer*

In [ ]:
%cd drive/MyDrive/DeepLearningFrom_Scratch/Book_1/ch07/

In [ ]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) # (data, channel, height, width)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # (9, 75) -> column =  3 * 5 * 5 number of filter elements

x2 = np.random.rand(10, 3, 7, 7) # 10 data
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) # (9, 75)

In [ ]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # filter deployment ->750 elements, reshpae(10, -1) -> 10 * 75 array
        out = np.dot(col, col_W) + self.b

        out = out.reshpae(N, out_h, out_w, -1).trasnpose(0, 3, 1, 2)

        return out


In [ ]:
#